In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import sys
import os
import shutil
sys.path.append("../../../")

In [3]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.algorithms.binarize.utils import *
from trailmet.datasets.classification import DatasetFactory

/opt/conda/envs/py117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models import ModelsFactory

In [5]:
import yaml
root = './'
import os
with open(os.path.join(root,"birealnet_cifar100.yaml"),'r') as stream:
    kwargs = yaml.safe_load(stream)

In [6]:
kwargs

{'batch_size': 256,
 'valid_size': 0.1,
 'num_train': 0,
 'epochs': 1,
 'optimizer': 'torch.optim.Adam',
 'lr': 0.001,
 'momentum': 0.9,
 'save_path': './save_path_resnet50_cifar100_fullbin',
 'data_path': '',
 'label_smooth': 0.1,
 'weight_decay': 0,
 'workers': 4,
 'device': 'cuda',
 'dataset': 'c100',
 'num_classes': 100,
 'num_fp': 0,
 'insize': 32,
 'wandb': True}

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


In [8]:
# Augmentation
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

valid_transform = transforms.Compose([
        transforms.Resize(36),
        transforms.CenterCrop(32),
        transforms.ToTensor()
    ])
test_transform = valid_transform

input_transforms = {
    'train': train_transform, 
    'val': valid_transform, 
    'test': test_transform}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

In [9]:
# Create Dataset
cifar100_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = "./",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.15,
                                        transform = input_transforms,
                                        target_transform = target_transforms
                                        )
# Define DataLoaders
train_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['train'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['train_sampler'],
        num_workers=kwargs['workers']
    )
val_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['val'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['val_sampler'],
        num_workers=kwargs['workers']
    )
test_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['test'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['test_sampler'],
        num_workers=kwargs['workers']
    )

dataloaders = {
        'train': train_loader100, 'val': val_loader100, "test": test_loader100
}


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [10]:
model = ModelsFactory.create_model(name = 'mobilenetv2', version='bireal', **kwargs)


In [11]:
algo = BirealNet(model, dataloaders, **kwargs)

wandb: Currently logged in as: animesh-007. Use `wandb login --relogin` to force relogin


In [12]:
algo.binarize()

Training BiRealNet Epoch [X] (X / X Steps) (batch time=X.Xs) (data time=X.Xs) (loss=X.X) (top1=X.X) (top5=X.X):   0%|| 0/167 [00:00<?, ?it/s]

learning_rate: 0.0


/opt/conda/envs/py117/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Training BiRealNet Epoch [0] (167 / 167 Steps) (batch time=0.24529s) (data time=0.00281s) (loss=4.80715) (top1=0.00000) (top5=0.00000): 100%|| 167/167 [00:15<00:00, 10.55it/s]
Validating BiRealNet Epoch [0] (30 / 30 Steps) (batch time=0.23393s) (loss=4.87406) (top1=1.31579) (top5=3.94737): 100%|| 30/30 [00:01<00:00, 19.08it/s]


 * Val acc@1 0.667 Val acc@5 4.160


Testing BiRealNet Epoch [0] (40 / 40 Steps) (batch time=0.16732s) (loss=4.75638) (top1=0.00000) (top5=6.25000): 100%|| 40/40 [00:01<00:00, 24.17it/s]

 * Test acc@1 0.660 Test acc@5 4.310
total training time = 0.0012819021145502726 hours
